# Project 1 Functions

-Ben and Jax

# Import libraries

In [13]:
import requests
import pandas as pd
import json
import csv
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

with open(r'C:\Users\bmcca\.secret\yelp_api.json') as f:
    keys = json.load(f)

client_id = keys['id']
yelp_key = keys['key']

# Request Business Data

## ƒ: yelp_request_pages

In [14]:
def yelp_request_pages(term, location, yelp_key, offset=0, verbose=False):
    '''Adapted from Yelp API Lab: https://github.com/BenJMcCarty/dsc-yelp-api-lab/tree/solution'''
    
    url = 'https://api.yelp.com/v3/businesses/search'

    headers = {
            'Authorization': 'Bearer {}'.format(yelp_key),
        }

    url_params = {
                    'term': term.replace(' ', '+'),
                    'location': location.replace(' ', '+'),
                    'limit': 50,
                    'offset': offset
                        }
    
    response = requests.get(url, headers=headers, params=url_params)
    
    if verbose == True:
        print(response)
        print(type(response.text))
        print(response.text[:1000])
        
    return response.json()

# Parsing and Prepping Data

## ƒ: parse_data

In [15]:
def parse_data(list_of_data):
    '''Adapted from Tyrell's code'''  

    # Create empty list to store results
    
    parsed_data = []
    
    # Loop through each business in the list of businesses
    # Add specific k:v pairs to a dictionary
    # These pairs will be used to build a DF afterwards
    
    for business in list_of_data:
        if 'price' not in business:
            business['price'] = np.nan
            
            # Verify that the "price" key is in the selected business dict
            
        details = {'name': business['name'],
                     'location': ' '.join(business['location']['display_address']),
                     'id': business['id'],
                     'alias': business['categories'][0]['alias'],
                     'title': business['categories'][0]['title'],
                     'rating': business['rating'],
                     'review_count': business['review_count'],
                     'price': business['price'],
                     'latitude': business['coordinates']['latitude'],
                     'longitude': business['coordinates']['longitude']
                    }

        parsed_data.append(details)

    # Create a DataFrame from the resulting list
    
    df_parsed_data = pd.DataFrame(parsed_data)
    
   
    return df_parsed_data

# Requesting, Parsing, and Saving Business Data

## ƒ: get_full_data

In [16]:
def get_full_data(term, location, yelp_key, file_name = 'data/wineries.csv'):
    '''Requests all results from Yelp API; saves as a .csv; and returns a DataFrame.'''
    blank_df = pd.DataFrame()
    blank_df.to_csv(file_name)
    
    # Process first request to Yelp API and calculate number of pages 
    results = yelp_request_offset(term, location, yelp_key, offset=0, verbose=False)
    num_pages = results['total']//50+1
    
    # Print out confirmation feedback
    print(f'For {term} and {location}: ')
    print(f"    Total number of results: {results['total']}.")
    print(f'    Total number of pages: {num_pages}.')
    
    # Create offset for further results and create empty list
    offset = 0
    parsed_results_dfs = []

    # Retrieves remaining pages
    for num in range(num_pages-1):
        try:
            results = yelp_request_offset(term, location, yelp_key, offset=offset, verbose=False)
            parsed_results = parse_data(results['businesses'])
            parsed_results_dfs.append(parsed_results)
            offset += 50
        except:
            print(f'Error on page {num}.')
            parsed_results_dfs.to_csv(file_name, mode='a')

    # Concatenate DataFrames and save to .csv
    df_concat = pd.concat(parsed_results_dfs, ignore_index=True)

    try:
        df_concat.to_csv(file_name)
        print(f'Saved to {file_name}.')
    except:
        print(f'Error, did not save.')
        
    return df_concat

## Cleaning Data

### ƒ: clean_data

In [17]:
def clean_data(existing_dataframe = None, raw_data = 'data/wineries_raw.csv'):
    '''- Requires data from either an existing dataframe or an existing .csv file
    - Takes raw business data from the Yelp API and filters for the top two
    aliases (focusing on "wineries" and "winetastingrooms").
    '''

    # Read in businesses
    df1 = pd.read_csv(raw_data, header = 1)

    alias_index = df1['alias'].value_counts()[:2].index
    print(alias_index)
    
    # Filtering rows based on condition

    df2 = df1[df1['alias'].isin(alias_index)]
    
    # Resetting index
    df2.reset_index(drop=True, inplace=True)
    
    # Save results
    df2.to_csv('data/wineries_cleaned.csv',index = False)
       
    print("Saved to 'data/wineries_cleaned.csv'")
    
    return df2

### ƒ: convert_price

In [18]:
def convert_price(dataframe):
    ''' - Requires a dataframe with the 'price' column elements being NaN, $, $$, or $$$.
    - Takes a pre-existing dataframe and adds a column to store the conversion from $ to an integer.'''
    
    # Converting $s to integers, then saving to new column.
    dataframe['price_converted'] = dataframe.loc[:,'price'].map({np.nan:0, '$':1, '$$':2, '$$$':3})
    
    # Saves results to new file
    dataframe.to_csv('data/wineries_price_converted.csv',index = False)
    
    return dataframe

# Request Business Reviews

## Obtain Business IDs

In [19]:
# df_saved = pd.read_csv("data/wineries_price_converted.csv")
# df_saved.reset_index(drop=True, inplace=True)

# # Slice out IDs, then save them to a list

# df_saved_id = df_saved['business id'].to_list()

# print(len(df_saved_id))
# print(type(df_saved_id))

## ƒ: yelp_request_reviews

In [20]:
def yelp_request_reviews(business_ID, yelp_key, verbose=False):
    '''- Requires 1x business ID, 1x Yelp API Key
    - Can specify if "verbose" is True for detailed API results (default False)
    - Yelp API call to obtain 3 reviews for a given business
    - Adapted from Yelp API Lab: https://github.com/BenJMcCarty/dsc-yelp-api-lab/tree/solution'''
    
    url = 'https://api.yelp.com/v3/businesses/'+ business_ID + '/reviews'

    headers = {
            'Authorization': 'Bearer {}'.format(yelp_key),
        }

    response = requests.get(url, headers=headers)

    if verbose == True:
        print(response)
        print(type(response.text))
        print(response.text[:1000])


    return response.json()

# Parse Reviews

## ƒ: parse_reviews

In [21]:
def parse_reviews(review):
    '''- Adapted from Tyrell's code
    - Requires a single business review from Yelp API
    - Loop through a single review and create a dictionary of Name, Rating,
    Text of the review, and Time Created'''  

    
    # Loop through each review in the list of reviews
    # Add specific k:v pairs to a dictionary      
    details = {
        'Reviewer Name': review['user']['name'],
        'Review Rating': review['rating'],
        'Review Text': review['text'],
        'Time Created': review['time_created']
        }


    # Create a DataFrame from the resulting dictionary
    
    df_parsed_reviews = pd.DataFrame.from_dict([details])
   
    return df_parsed_reviews

# Pull remaining reviews

## ƒ: get_all_reviews

In [22]:
def get_all_reviews(list_of_biz_ids, yelp_key, file_name = 'data/reviews_raw.csv'):
    '''Requests all review results for given business IDs from Yelp API; \
    saves as 'data/reviews_raw.csv'; and returns a DataFrame.'''
    
    # Create a starter empty DataFrame and save to .csv to store data.    
    blank_df = pd.DataFrame(columns= ['Reviewer Name', 'Review Rating', 
                                      'Review Text', 'Time Created', 
                                      'id'])
    blank_df.to_csv(file_name, index = False)
        
    for i in list_of_biz_ids:
        try:
            
            # Process API request for 3 reviews per business:
            raw_reviews = get_reviews(i, yelp_key)

            for review in raw_reviews['reviews']:
                

                # From results, take values from "Businesses" key and save
                parsed_reviews = parse_reviews(review) 

                parsed_reviews['id'] = i
                
                # H2: save results to df
                parsed_reviews.to_csv(file_name, mode='a', index = False,
                                      header = False)

        except:
            # If error, print where the error happens
            print(f'Error on page {num}.')
            # Then save the results so far to the .csv
            parsed_reviews.to_csv(file_name, mode='a', index = False, 
                                  header = False)

    try:
        reviews1 = pd.read_csv(file_name)
        return reviews1
    except:
        return parsed_reviews